In [1]:

%load_ext autoreload
%autoreload 2

import sys
sys.path.insert(0,'/app')
sys.path.insert(0,'..')

In [2]:

from audio.redis import *
from audio.audio import *
# from audio.paths import *
# from audio.tools import *
# from audio.psql import *

from pathlib import Path
import json

import uuid

import pandas as pd

import redis
import asyncio
import redis.asyncio as aioredis
from dataclasses import dataclass
import numpy as np

/tmp/ipykernel_389/4230643358.py:12: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [3]:
# import uuid
# client_id = str(uuid.uuid4());client_id

In [4]:

redis_inner_client = await get_inner_redis()
redis_stream_client = await get_stream_redis()
client_id = '851f343e-4954-4f0a-8835-9664cc91c181'

In [5]:


@dataclass
class Notification:
    channel_name: str
    redis_client: aioredis.Redis

    async def initialize(self):
     #   self.redis_client = await get_inner_redis()
        async with self.redis_client.pubsub() as sub:
            await sub.subscribe(self.channel_name)

    async def wait(self, msg):
        async with self.redis_client.pubsub() as sub:
            await sub.subscribe(self.channel_name)
            while True:
                r = await sub.get_message(ignore_subscribe_messages=True)
                if r and r['data'] == msg:
                    print('done')
                    break
                await asyncio.sleep(0)  # Yield control

In [6]:

diarize_ready_notification = Notification('DiarizeReady',redis_inner_client)
await diarize_ready_notification.initialize()
transcribe_ready_notification = Notification('TranscribeReady',redis_inner_client)
await transcribe_ready_notification.initialize()

In [7]:

async def diarize(audio_data, client_id, audio_name, shift):
    audio = Audio(chunk_name=audio_name, redis_client=redis_inner_client, data=audio_data)
    await audio.save()
    await redis_inner_client.lpush('Audio2DiarizeQueue', f'{audio_name}:{client_id}')
    await diarize_ready_notification.wait(audio_name)
    diarization = Diarisation(audio_name, redis_inner_client)
    await diarization.get()
    df = pd.DataFrame(diarization.data)
    df['silence'] = df['start']-df['end'].shift()
    df['speaker_change'] = df['speaker'] != df['speaker'].shift()
    df['len'] = df['end'] - df['start']
    df = df[df['len'] > 0.5]
    df['speaker_change'] = np.where(df['silence']>2,True,df['speaker_change'])

    df['speaker_change'] = df['speaker_change'].cumsum()
    df = df.groupby('speaker_change').agg({'speaker': 'first', 'start': 'first', 'end': 'last'})
    df['conv_start'] = df['start'] + shift
    df['conv_end'] = df['end'] + shift
    return df.to_dict('records')

In [8]:

async def transcribe(diarization_result,connection_id,audio_name):
    for segment in diarization_result:
        segment['connection_id'] = connection_id
        segment['name'] = str(uuid.uuid4())
        segment['audio_id'] = audio_name
        await redis_inner_client.lpush('TranscribeQueue', json.dumps(segment))
        await transcribe_ready_notification.wait(segment['name'])
       # wait_notification(transcribe_ready_notification,segment['name'])
        transcription = Transcript(segment["name"],redis_inner_client)
        await transcription.get()
        segment['transcription'] = transcription.data
        await redis_inner_client.lpush(f'Segment:{connection_id}', json.dumps(segment))
        await redis_inner_client.publish(f'SegmentReady', segment["name"])

In [9]:
async def get_next_chunk_start(diarization_result, length):
    if len(diarization_result)>0:
        last_speech = diarization_result[-1]
        if round(last_speech['end']) == length:
            return last_speech['conv_start']
        

        else: return last_speech['conv_end']



    else: return None

In [10]:
#from file

#path = Path('/app/audio').ls()[1]
path = Path('/app/testdata/david_audio.webm')
audio_slicer =await AudioSlicer.from_file(path,'webm')
connection_id = path.name

In [11]:
async def get_next_chunk_start(diarization_result, length):
    if len(diarization_result)>0:
        last_speech = diarization_result[-1]
        if last_speech['end']+0.1 > length:
            return last_speech['conv_start']
        
        else: 
            return last_speech['conv_end']



    else: return None

In [12]:
start = 0
length = 60
end = start+length

In [13]:
connection_id = 'david_audio.webm'#'david_audio.webm'

In [14]:
while True:
    audio_data = await audio_slicer.export_data(start,end)
    audio_name = str(uuid.uuid4())
    audio = Audio(chunk_name=audio_name, redis_client=redis_inner_client, data=audio_data)
    await audio.save()
    diarization_result = await diarize(audio_data,client_id,audio_name,start)
    await transcribe(diarization_result,connection_id,audio_name)
    start_ = await get_next_chunk_start(diarization_result,length)
    start = start_ if start_ else start+length
    if end>audio_slicer.audio.duration_seconds:break
    end+=length

done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done


In [ ]:
print(start,end)

0.46689303904923596 120


In [ ]:
audio_slicer.slice(start,end)

In [ ]:
print(start,end)

300.0339558573854 420


In [ ]:
audio_slicer.slice(295,305)

In [ ]:
start

21.264855687606115

In [ ]:
diarization_result

[{'speaker': 'f320b84b-1150-4904-a8af-54ffbcd9ba4c',
  'start': 0.46689303904923596,
  'end': 21.264855687606115,
  'conv_start': 0.46689303904923596,
  'conv_end': 21.264855687606115,
  'connection_id': 'david_audio.webm',
  'name': 'b070193d-87dd-46ce-9358-1df5c34ef6c8',
  'audio_id': '97a9e0c2-7f74-4aa5-8489-d6487e3c79be',
  'transcription': [{'start': 0.0,
    'end': 3.68,
    'text': ' but this life cycle is indicative only, right?'},
   {'start': 3.68,
    'end': 6.6000000000000005,
    'text': ' So in other words, you could take any life cycle'},
   {'start': 6.6000000000000005,
    'end': 8.22,
    'text': " and do what I'm gonna do here."},
   {'start': 8.22,
    'end': 13.06,
    'text': ' So initiation, we typically have discovery,'},
   {'start': 16.38, 'end': 20.8, 'text': ' design, build,'}]}]

In [ ]:

start = 0
length = 20
end = start+length
while True:
    audio_data = await audio_slicer.export_data(start,end)
    audio_name = str(uuid.uuid4())
    audio = Audio(chunk_name=audio_name, redis_client=redis_client, data=audio_data)
    #await audio.fix()
    await audio.save()
    diarization_result = await diarize(audio_data,client_id,audio_name,start)
    await transcribe(diarization_result,connection_id,audio_name)
    start_ = await get_next_chunk_start(diarization_result,length)
    start = start_ if start_ else start+length
    end+=length

done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done


In [ ]:
while True:
    audio_data = await audio_slicer.export_data(start,start+len)
    audio_name = str(uuid.uuid4())
    diarization_result = await diarize(audio_data,client_id,audio_name,start)
    await transcribe(diarization_result,connection_id,audio_name)

    start+=len

    

done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done


CouldntDecodeError: Decoding failed. ffmpeg returned error code: 1

Output from ffmpeg/avlib:

ffmpeg version 4.4.2-0ubuntu0.22.04.1 Copyright (c) 2000-2021 the FFmpeg developers
  built with gcc 11 (Ubuntu 11.2.0-19ubuntu1)
  configuration: --prefix=/usr --extra-version=0ubuntu0.22.04.1 --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --arch=amd64 --enable-gpl --disable-stripping --enable-gnutls --enable-ladspa --enable-libaom --enable-libass --enable-libbluray --enable-libbs2b --enable-libcaca --enable-libcdio --enable-libcodec2 --enable-libdav1d --enable-libflite --enable-libfontconfig --enable-libfreetype --enable-libfribidi --enable-libgme --enable-libgsm --enable-libjack --enable-libmp3lame --enable-libmysofa --enable-libopenjpeg --enable-libopenmpt --enable-libopus --enable-libpulse --enable-librabbitmq --enable-librubberband --enable-libshine --enable-libsnappy --enable-libsoxr --enable-libspeex --enable-libsrt --enable-libssh --ena